### 1. The Morse Potential (energies and wavefunctions)
This potential is given by 

$$V(R)=D_e \{1-exp[-(R-R_e)a]\}^2$$

Plot this for two cases (use the same frame of x and y.

a) $D_e$ = 4.7 𝑒𝑉, $R_e$ = 1.4 bohr, $a$ = 1.0 bohr

b) $D_e$ = 2.5 𝑒𝑉, $R_e$ = 4.4 bohr, $a$= 2.0 bohr
    
c) The Hamiltonian is $-\frac{\hbar^2}{2\mu}\frac{d^2}{d R^2} +V(R)$ and the PIB basis is the usual one but in R and not x. For case a), use a PIB basis (box 0 to L) to obtain the first 5 energy eigenvalues and wavefunctions converged to at least three significant figures.  Use the reduced mass of the H$_2$ molecule, which is $m_\mathrm{p}$/2.(Don't forget to convert it to the mass in atomic units, where the unit is $m_e$!). Thinks about how to locate the box. One approach is to set the center the box at $R_e$. Maybe there is a better location. The basisw should have no less than 20 PIB functions.

In [ ]:
# Insert your answer here
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import scipy
import scipy.special
import scipy.constants as cst
import scipy.integrate

# This class gives us objects that can be passed around without having to specify the parameters every time.
class MorsePotential:
    def __init__(self, D_e, R_e, a):
        self.D_e = D_e # eV
        self.R_e = R_e # a_0
        self.a = a  # Inverse a_0
        self.D_e_eh = D_e*cst.physical_constants['electron volt-hartree relationship'][0] # Hartree
        self.omega = np.sqrt(2*self.D_e_eh*self.a**2/MU)
        self.omega_xe = self.a**2/(2*MU)
        self.xe = self.omega_xe/self.omega
        self.get_exact_eigvals()

    def get_exact_eigvals(self, start=0, end=4):
        self.eigvals = np.array([self.omega*(_+0.5) - self.omega_xe*(_+0.5)**2 for _ in range(start,end+1)])
        self.eigvals_ev = self.eigvals/cst.physical_constants['electron volt-hartree relationship'][0]
        
    def get_pot(self, R):
        return self.D_e_eh*(1 - (np.exp(-self.a*(R-self.R_e))))**2
    def plot(self, R_lower, R_upper, ylim):
        f,ax = plt.subplots(figsize=(8,8))
        R = np.linspace(R_lower, R_upper, 10000)
        V_R = self.get_pot(R, self.D_e, self.a, self.R_e)
        ax.plot(R, V_R)
        ax.set_xlabel(r'$R$ / $a_0$')
        ax.set_ylabel(r'$V(R)$ / eV')
        ax.set_ylim(ylim)
        ax.set_title(r'Morse potential, $D_e={D_e}$ eV, $R_e={R_e}$ $a_0$, $a={a}$ $a_0$'.format(D_e=self.D_e, R_e=self.R_e, a=self.a))

# These are the functional versions of the class above. This is more convenient for interactive plotting.
# [todo]: Interactive plotting.
        
def morse_potential(R, D_e, R_e, a):
    return D_e*(1 - (np.exp(-a*(R-R_e))))**2

def plot_morse_potential(R_lower, R_upper, D_e, a, R_e, ylim):
    f,ax = plt.subplots(figsize=(8,8))
    R = np.linspace(R_lower, R_upper, 10000)
    V_R = morse_potential(R, D_e, a, R_e)
    ax.plot(R, V_R)
    ax.set_xlabel(r'$R$ / $a_0$')
    ax.set_ylabel(r'$V(R)$ / eV')
    ax.set_ylim(ylim)
    ax.set_title(r'Morse potential, $D_e={D_e}$ eV, $R_e={R_e}$ $a_0$, $a={a}$ $a_0$'.format(D_e=D_e, R_e=R_e, a=a))
plot_morse_potential(0,10,4.7,1.4,1.0,[-1,10])
#plot_morse_potential(0,10,2.5,4.4,2.0,[-1,10])

In [ ]:
#part c
import scipy as scipy
import numpy as np
from math import sin
from scipy.integrate import quad
from numpy import linalg as LA
L = 2.8
Dimension = 30
mu = 920

In [ ]:
R_e = 1.4
a = 1.0
D_e = 4.7/27.2113961
import math
def v(x,m,n):
    return sin(m * math.pi * x / L) * D_e*(1 - (np.exp(-a*(x-R_e))))**2 * sin(n * math.pi * x / L)

In [ ]:
lb = 1.4 - L/2
ub = 1.4 + L/2
def H(Dimension,L):
    A = np.zeros(shape=(Dimension,Dimension))

    for m in range(1, Dimension + 1):
        for n in range(1, Dimension + 1):
            mu = 920
            vint, verror = quad(v, lb, ub, args=(m, n))
            result = vint 
            if m == n:
                result =result+(n**2 * np.pi**2)/(mu * L**3)
            A[m-1][n-1] = result

    A = np.matrix(A)
    A = A.round(3)
    return A

In [ ]:
H_matrix = H(Dimension,L)
#print(H_matrix)

In [ ]:
eigenvalue, eigenvector = LA.eigh(H_matrix)
print(eigenvalue)

In [ ]:
from math import sqrt
%matplotlib inline
def psi(x, n):
    return sqrt(2 / L) * np.sin(n * math.pi * (x + L / 2) / L)
x = np.arange(-L/2,L/2,0.001)   # start,stop,step
# plot each psi_v in different graphs
for i in range(6):
    eigenfunction = 0
    for index, vector in enumerate(np.array(eigenvector[:, i])):
        eigenfunction += psi(x, index + 1) * vector
    plt.plot(x, eigenfunction+2*i, label="v = " + str(i))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()

### 2. Solve Morse potential with Numerov method

```NumerovHO-class.ipynb``` in this directory is the example code provided by the instructor to solve the **Harmonic Oscillator** problem with Numerov method. 
We assume that $\mu=0.948087 m_p$, where $m_p$ is the mass of proton, $\omega=0.017$.
We fix the number of discrete points to be 1000 in the range $x\in[-2,2]$

a) Assume that the potential has the following form with anharmonic terms $x^3$ and $x^4$, solve the problem and obtain the energy levels <0.2 Hartree. Compare the energy levels with the harmonic case. 

V(x)=$\frac{1}{2}\mu\omega^2(x^2-0.4x^3+0.007x^4)$

b) How does the energy difference between two neighbhoring vibration states change as quantum number $v$ increases?




In [ ]:
# Insert your answer here

### 3. Fit real diatomic molecule vibration spectrum

Read in the data set for vibrational energies in cm$^{-1}$ from the file 'spectrum.csv' in the current directory.

The goal is to fit $E_v$ to the following (textbook) expression 

$$E=(v+1/2)\hbar\omega - (v+1/2)^2\hbar\omega x_e$$

Use a least squre fitting to determine $\omega$ and $\omega x_e$

In [ ]:
# Insert your answer here

### 4. Vibrating rotor problem

For diatomic molecules, including rotation modifies the Hamiltonian in a simple way, namely

$$\hat{H}=-\frac{\hbar^2}{2\mu}\frac{d^2}{dR^2} +\frac{\hbar^2j(j+1)}{2\mu R^2} + V(R),\quad j=0,1,2,\dots $$
Here $V(R)$ is the Morse potential defined in Problem 1. 

a) Replot the potential for case a) above in problem 1 for j = 2, 4, 8
And locate the minima for these. Note the sum of $V(R)$ and the centrifugal potential is denoted as the effective potential 

$$V_\mathrm{eff}=\frac{\hbar^2j(j+1)}{2\mu R^2} + V(R)$$

These minima may need to be determined numerically. Do that for each j.

b) Determine the first five eigenvalues for these three j values. Summarize the results in a
table and include the results from Problem 1, which are for $j = 0$.

c) You have just solved the “vibrating rotor” problem for diatomic molecules exactly.

### 5. Fits spectrum using eigenvalues from Problem 1

a) Use the eigenvalues for the Morse potential

$D_e$ = 4.7 𝑒𝑉, $R_e$ = 1.4 bohr, $a$ = 1.0 bohr, and j = 0

in Problem 1, to perform the fits $$E_v=(v+1/2)\hbar\omega - (v+1/2)^2\hbar\omega x_e$$ for $v$ in the range 0 to 5 and then again in the range 0 to 12.



b) For $v = 1$ determine eigenvalues for $j = 0, 2, 4, 8,$ i.e., $E_{v=1,j}$ Fit these using the expression

$$E_{v=1,j}=Bj(j+1)+D[j(j+1)]^2$$

That is determine B and D using linear least squares.